In [1]:
from pydantic import BaseModel, Field, HttpUrl
from typing import List
import configparser

# Define Pydantic models for each config section
class YouTrackConfig(BaseModel):
    Token: str
    BaseUrl: str

class VsemRabotaConfig(BaseModel):
    ProjectId: str
    WBTeam: List[str] = Field(default_factory=list)
    Name: str

# Main configuration model that includes both sections
class AppConfig(BaseModel):
    YouTrack: YouTrackConfig
    VsemRabota: VsemRabotaConfig

# Parse the INI file
def parse_config_to_model(config_file: str) -> AppConfig:
    config = configparser.ConfigParser()
    config.read(config_file)
    
    # YouTrack section
    youtrack_token = config.get('YouTrack', 'Token').strip("'")
    youtrack_base_url = config.get('YouTrack', 'BaseUrl')
    
    # VsemRabota Config section
    project_id = config.get('VsemRabota Config', 'ProjectId')
    name = config.get('VsemRabota Config', 'Name')
    wb_team = config.get('VsemRabota Config', 'WBTeam').split(', ')
    
    # Create instances of the models
    youtrack_config = YouTrackConfig(Token=youtrack_token, BaseUrl=youtrack_base_url)
    vsem_rabota_config = VsemRabotaConfig(ProjectId=project_id, WBTeam=wb_team, Name=name)
    
    # Aggregate into the main config model
    app_config = AppConfig(YouTrack=youtrack_config, VsemRabota=vsem_rabota_config)
    
    return app_config

In [2]:
config_file = 'yt.ini'  # Update this path
config_model = parse_config_to_model(config_file)
print(config_model)

YouTrack=YouTrackConfig(Token='perm:VmFsZW50aW5fTWFyY2h1aw==.NjktMzE1.IgVAwvLdgn8H1dpNCX2FOS9T0Yhdpn', BaseUrl='https://youtrack.wildberries.ru') VsemRabota=VsemRabotaConfig(ProjectId='77-167', WBTeam=['Valentin_Marchuk', 'Liventsev_sergei'], Name='VsemRabota')


In [ ]:
import requests

headers = {
    'Authorization': f'Bearer {config_model.YouTrack.Token}',
    'Content-Type': 'application/json',
}

search_query = 'Business Line:HR   State: Resolved sort by: {Threat Score} desc'

response = requests.get(f'{config_model.YouTrack.Token}/api/issues?query={search_query}', headers=headers)

In [47]:
import requests
from urllib.parse import urlencode

class YouTrackAPIError(Exception):
    """Exception raised for errors in the YouTrack API."""
    def __init__(self, status_code, message):
        super().__init__(f"HTTP {status_code}: {message}")
        self.status_code = status_code
        self.message = message

class YouTrackClient:
    def __init__(self, config):
        self.config = config
        self.headers = {
            'Authorization': f'Bearer {self.config.Token}',
            'Content-Type': 'application/json',
        }

    #     headers = {
    #     'Authorization': f'Bearer {api_token}',
    #     'Content-Type': 'application/json',
    # }

    def get(self, endpoint: str, params: dict = None) -> dict:
        """Sends a GET request to a specified endpoint with optional query parameters."""
        url = f"{self.config.BaseUrl}{endpoint}"
        # param_str = '&'.join([f'{key}={value}' for key, value in params.items()])
        if params:
            url += f"?{urlencode(params)}"
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            raise YouTrackAPIError(response.status_code, response.text) from e

    def post(self, endpoint: str, data: dict = None) -> dict:
        """Sends a POST request to a specified endpoint with optional JSON data."""
        url = f"{self.config.BaseUrl}{endpoint}"
        try:
            response = requests.post(url, json=data, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            raise YouTrackAPIError(response.status_code, response.text) from e

    def get_issues(self, search_query: str, additional_params: dict = None) -> dict:
        """
        Retrieves issues from YouTrack based on a search query and additional parameters.

        :param search_query: A string representing the search query for filtering issues.
        :param additional_params: Optional dictionary of additional query parameters.
        :return: A dictionary containing the API response with issues.
        """
        params = {'query': search_query}
        if additional_params:
            params.update(additional_params)
        
        return self.get('/api/issues', params=params)

    def get_issue(self, issue_id, fields=None):
        issue_fields = [
            'id', 'idReadable', 'summary', 'description', 'reporter(id,login,fullName)',
            'customFields($type,id,projectCustomField($type,id,field($type,id,name)),value($type,avatarUrl,buildLink,color(id),fullName,id,isResolved,localizedName,login,minutes,name,presentation,text))',
            'linkType(name,sourceToTarget,targetToSource)',
            'issues(id,idReadable,summary)'
        ]
        
        if fields is None:
            fields = 'id,idReadable,summary,description,customFields(id,projectCustomField(field(name))),linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)'
            fields = ','.join(issue_fields)

        params = {
            'fields': fields
        }

        return self.get(f'/api/issues/{issue_id}', params=params)

    def get_field(self, issue_id, field_id):
        params = {'fields': 'id,projectCustomField(id,field(id,name)),value(id,isResolved,localizedName,name)'}
        return self.get(f'/api/issues/{issue_id}/customFields/{field_id}', params=params)

    def get_issue_history(self, issue_id):
        'api/issues/SP-32/activities'

        params = {
            'categories': 'CustomFieldCategory,CommentsCategory',
            'fields': 'author(name,login),timestamp,target(text),added(name),removed(name)'
        }
        return self.get(f'/api/issues/{issue_id}/activities', params=params)

    def get_projects(self, additional_params: dict = None):
        leader_fields = ['login', 'name', 'id']
        createdby_fields = ['login', 'name', 'id']
        
        params = {'fields': 'id,name,shortName,createdBy(login,name,id),leader(login,name,id),key'}
        if additional_params:
            params.update(additional_params)
        
        return self.get('/api/admin/projects', params=params)
    

In [32]:
# youtrack_config = YouTrackConfig(Token='your_token_here', BaseUrl='https://your-instance.myjetbrains.com/youtrack')
youtrack_client = YouTrackClient(config=config_model.YouTrack)
        

In [ ]:
resp = youtrack_client.get_issue('VR-2934')

# resp['customFields']
resp


In [88]:
def get_customfield_value(customfield, issue):
    return next(r['value'] for r in issue['customFields'] if r['projectCustomField']['field']['name'] == customfield)

def get_issue_authority(issue):
    assignee = get_customfield_value('Assignee', issue)
    reviewer = get_customfield_value('Reviewer [HR]', issue)
    product = get_customfield_value('Product', issue)
    reporter = resp['reporter']['fullName']

    return {
        'assignee': assignee['fullName'] if assignee else None,
        'reviewer': reviewer['fullName'] if reviewer else None,
        'products': [p['name'] for p in product],
        'reporter': reporter
    }

In [ ]:
import pandas as pd

file_path = r'C:\Users\MGroup\Downloads\VULN tokens.csv'
df = pd.read_csv(file_path)

issues = list(dict.fromkeys(df['Ссылка на youtrack'].values.tolist()))
issues = [ i.replace('https://youtrack.wildberries.ru/issue/', '') for i in issues]
issues

In [ ]:
df

In [ ]:
issues_authority = []
for id in issues:
    issue = youtrack_client.get_issue(id)
    if issue is None:
        print(f'Issue {id} was not found.')
        continue
    issue_authority = get_issue_authority(issue)
    issues_authority.append({
         'issue_id': id,
        **issue_authority
    })


issues_authority

In [95]:
df['issue_id'] = df['Ссылка на youtrack'].str.extract(r'issue/(VR-\d+)')

json_df = pd.DataFrame(issues_authority)
merged_df = pd.merge(df, json_df, on="issue_id", how="left")

merged_df.to_csv('vulns.csv')

In [104]:
merged_df['products'].apply(assign_team)

0        WB Team
1        WB Team
2        WB Team
3        WB Team
4       ATS Team
         ...    
72      ATS Team
73    Empty Team
74    Empty Team
75    Empty Team
76    Empty Team
Name: products, Length: 77, dtype: object

In [ ]:

# Define a function to determine the team based on the product
def assign_team(products):
    if any(product in ['admin', 'vsemrabota', 'wbteam'] for product in products):
        return 'WB Team'
    elif 'ats' in products:
        return 'ATS Team'
    else:
        return 'Empty Team'

# Apply the function to create a new 'Team' column
merged_df['Team'] = merged_df['products'].apply(assign_team)

# Group by 'Team' and then by 'issue_id', and collect tokens
grouped_data = merged_df.groupby(['Team', 'issue_id'])['Секрет'].apply(list)

output_format = {}
for (team, issue_id), tokens in grouped_data.items():
    if team not in output_format:
        output_format[team] = []
    output_format[team].append((issue_id, tokens))

# Print in the desired format
for team, issues in output_format.items():
    print(f"{team}:")
    for issue_id, tokens in issues:
        print(f"{issue_id}")
        for token in tokens:
            print(f"- {token}")
    print("\n")

In [108]:
merged_df.to_csv('vulns.csv')

In [20]:
issues = youtrack_client.get('/api/issues', params={'query': 'Business Line:HR   State: Resolved sort by: {Threat Score} desc'})
print(issues)

[{'id': '92-715620', '$type': 'Issue'}, {'id': '92-730734', '$type': 'Issue'}, {'id': '92-887120', '$type': 'Issue'}, {'id': '92-732032', '$type': 'Issue'}, {'id': '92-779165', '$type': 'Issue'}, {'id': '92-860643', '$type': 'Issue'}, {'id': '92-778840', '$type': 'Issue'}, {'id': '92-690675', '$type': 'Issue'}, {'id': '92-690639', '$type': 'Issue'}, {'id': '92-690615', '$type': 'Issue'}, {'id': '92-721263', '$type': 'Issue'}, {'id': '92-708541', '$type': 'Issue'}, {'id': '92-722135', '$type': 'Issue'}, {'id': '92-584660', '$type': 'Issue'}, {'id': '92-803271', '$type': 'Issue'}, {'id': '92-803320', '$type': 'Issue'}, {'id': '92-672794', '$type': 'Issue'}, {'id': '92-686096', '$type': 'Issue'}, {'id': '92-685816', '$type': 'Issue'}, {'id': '92-729859', '$type': 'Issue'}, {'id': '92-732665', '$type': 'Issue'}, {'id': '92-730768', '$type': 'Issue'}, {'id': '92-693085', '$type': 'Issue'}, {'id': '92-730682', '$type': 'Issue'}, {'id': '92-777825', '$type': 'Issue'}, {'id': '92-1029406', '$t

In [ ]:
youtrack_client.get_issues('Business Line:HR   State: Resolved sort by: {Threat Score} desc')

In [ ]:
projects = youtrack_client.get_projects(
    {'$top': '100', '$skip': '300'}
)
projects

In [44]:
resp = youtrack_client.get_issues('Reviewer [HR]: Valentin_Marchuk, Vansevich.E, Mikhail_Shogin, grigorev.mark , Vitaliy_Guselnikov, aleksandrov.e23, krupenko.ilya, mustafetov.n , goncharov.v38, korolev.artem14  and State: Review ')
resp

query=Reviewer [HR]: Valentin_Marchuk, Vansevich.E, Mikhail_Shogin, grigorev.mark , Vitaliy_Guselnikov, aleksandrov.e23, krupenko.ilya, mustafetov.n , goncharov.v38, korolev.artem14  and State: Review 


[{'id': '92-1177453', '$type': 'Issue'},
 {'id': '92-1106073', '$type': 'Issue'},
 {'id': '92-1177548', '$type': 'Issue'},
 {'id': '92-1101380', '$type': 'Issue'},
 {'id': '92-975486', '$type': 'Issue'},
 {'id': '92-1072736', '$type': 'Issue'},
 {'id': '92-1072253', '$type': 'Issue'},
 {'id': '92-1022092', '$type': 'Issue'},
 {'id': '92-1136199', '$type': 'Issue'},
 {'id': '92-1078352', '$type': 'Issue'},
 {'id': '92-1078349', '$type': 'Issue'},
 {'id': '92-1074757', '$type': 'Issue'},
 {'id': '92-1082991', '$type': 'Issue'},
 {'id': '92-1039379', '$type': 'Issue'},
 {'id': '92-1182121', '$type': 'Issue'},
 {'id': '92-1171940', '$type': 'Issue'},
 {'id': '92-1143505', '$type': 'Issue'},
 {'id': '92-1116084', '$type': 'Issue'},
 {'id': '92-1152478', '$type': 'Issue'},
 {'id': '92-1100264', '$type': 'Issue'},
 {'id': '92-944109', '$type': 'Issue'},
 {'id': '92-884017', '$type': 'Issue'},
 {'id': '92-884024', '$type': 'Issue'},
 {'id': '92-887370', '$type': 'Issue'},
 {'id': '92-1074057',

In [45]:
review_items = []

for item in resp:
    print(item['id'])
    issue = youtrack_client.get_issue(item['id'])
    history = youtrack_client.get_issue_history(issue["idReadable"])
    
    latest_review_time, hours_passed = get_latest_review_time_and_passed_hours(history)
    
    reviewer = next(field['value']['login'] for field in issue['customFields'] if field['projectCustomField']['field']['name'] == 'Reviewer [HR]')
    
    review_items.append({
        'reviewer': reviewer,
        'issue_id': issue["idReadable"],
        'summary': issue["summary"],
        'latest_review_time': latest_review_time,
        'hours_passed': hours_passed
    })

92-1177453
fields=id,idReadable,summary,description,reporter(id,login,fullName),customFields($type,id,projectCustomField($type,id,field($type,id,name)),value($type,avatarUrl,buildLink,color(id),fullName,id,isResolved,localizedName,login,minutes,name,presentation,text)),linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)
categories=CustomFieldCategory,CommentsCategory&fields=author(name,login),timestamp,target(text),added(name),removed(name)


C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\1595032851.py:15: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  latest_review_time = datetime.utcfromtimestamp(latest_review_timestamp_seconds)
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\1595032851.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow()


92-1106073
fields=id,idReadable,summary,description,reporter(id,login,fullName),customFields($type,id,projectCustomField($type,id,field($type,id,name)),value($type,avatarUrl,buildLink,color(id),fullName,id,isResolved,localizedName,login,minutes,name,presentation,text)),linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)
categories=CustomFieldCategory,CommentsCategory&fields=author(name,login),timestamp,target(text),added(name),removed(name)
92-1177548
fields=id,idReadable,summary,description,reporter(id,login,fullName),customFields($type,id,projectCustomField($type,id,field($type,id,name)),value($type,avatarUrl,buildLink,color(id),fullName,id,isResolved,localizedName,login,minutes,name,presentation,text)),linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)
categories=CustomFieldCategory,CommentsCategory&fields=author(name,login),timestamp,target(text),added(name),removed(name)
92-1101380
fields=id,idReadable,summary,description,reporter(id,

In [17]:
youtrack_client.get_field('VR-8939', '86-578')

{'value': {'name': 'Шогин Михаил Михайлович',
  'id': '24-4442',
  '$type': 'User'},
 'id': '86-578',
 'projectCustomField': {'field': {'name': 'Reviewer [HR]',
   'id': '42-918',
   '$type': 'CustomField'},
  'id': '86-578',
  '$type': 'UserProjectCustomField'},
 '$type': 'SingleUserIssueCustomField'}

In [42]:
issue = youtrack_client.get_issue('92-1177453')
history = youtrack_client.get_issue_history(issue["idReadable"])

latest_review_time, hours_passed = get_latest_review_time_and_passed_hours(history)

reviewer = next(field['value']['login'] for field in issue['customFields'] if field['projectCustomField']['field']['name'] == 'Reviewer [HR]')

item = {
    'reviewer': reviewer,
    'issue_id': issue["idReadable"],
    'summary': issue["summary"],
    'latest_review_time': latest_review_time,
    'hours_passed': hours_passed
}
# print(f'[{issue["idReadable"]}] {issue["summary"]} {reviewer}')

item

fields=id,idReadable,summary,description,reporter(id,login,fullName),customFields($type,id,projectCustomField($type,id,field($type,id,name)),value($type,avatarUrl,buildLink,color(id),fullName,id,isResolved,localizedName,login,minutes,name,presentation,text)),linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)
categories=CustomFieldCategory,CommentsCategory&fields=author(name,login),timestamp,target(text),added(name),removed(name)


C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\1595032851.py:15: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  latest_review_time = datetime.utcfromtimestamp(latest_review_timestamp_seconds)
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\1595032851.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow()


{'reviewer': 'Valentin_Marchuk',
 'issue_id': 'VR-8816',
 'summary': '[BE] Научиться запускать локально crm-api',
 'latest_review_time': datetime.datetime(2024, 3, 11, 20, 10, 15, 607000),
 'hours_passed': 1.4195444794444443}

In [ ]:
youtrack_client.get_issue('VR-8939')

In [ ]:
history = youtrack_client.get_issue_history('VR-8816')

history

In [37]:
activity_log = history

In [40]:
from datetime import datetime, timedelta

def get_latest_review_time_and_passed_hours(activity_log):
    # There was an error due to the first item in the log having 'added' as an integer. Let's correct this and process again.
    latest_review_timestamp = None
    for activity in activity_log:
        if 'added' in activity and isinstance(activity['added'], list) and any('name' in d and d['name'] == 'Review' for d in activity['added']):
            latest_review_timestamp = activity['timestamp']
    
    # Convert the timestamp from milliseconds to seconds
    latest_review_timestamp_seconds = latest_review_timestamp / 1000 if latest_review_timestamp else None
    
    # Calculate the hours passed from the latest status change to "Review" until now
    if latest_review_timestamp_seconds:
        latest_review_time = datetime.utcfromtimestamp(latest_review_timestamp_seconds)
        current_time = datetime.utcnow()
        hours_passed = (current_time - latest_review_time).total_seconds() / 3600
    else:
        hours_passed = None
    
    return latest_review_time, hours_passed

get_latest_review_time_and_passed_hours(activity_log)

C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\1595032851.py:15: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  latest_review_time = datetime.utcfromtimestamp(latest_review_timestamp_seconds)
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\1595032851.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow()


(datetime.datetime(2024, 3, 11, 20, 10, 15, 607000), 1.3813523625000002)

In [48]:
review_items

[{'reviewer': 'Valentin_Marchuk',
  'issue_id': 'VR-8816',
  'summary': '[BE] Научиться запускать локально crm-api',
  'latest_review_time': datetime.datetime(2024, 3, 11, 20, 10, 15, 607000),
  'hours_passed': 1.4520275355555554},
 {'reviewer': 'Vansevich.E',
  'issue_id': 'VR-8146',
  'summary': '[BE] Написать клиент для WB Basket и подключить к HR Scans',
  'latest_review_time': datetime.datetime(2024, 3, 11, 19, 25, 24, 315000),
  'hours_passed': 2.2000160105555557},
 {'reviewer': 'Mikhail_Shogin',
  'issue_id': 'HRWEB-2378',
  'summary': '[ARCH] WB Basket. Описание архитектуры',
  'latest_review_time': datetime.datetime(2024, 3, 11, 18, 23, 13, 427000),
  'hours_passed': 3.236877321388889},
 {'reviewer': 'Valentin_Marchuk',
  'issue_id': 'VR-8099',
  'summary': '[BE] Локальное развертывание сервисов crm-api',
  'latest_review_time': datetime.datetime(2024, 2, 6, 17, 32, 54, 681000),
  'hours_passed': 820.076044315},
 {'reviewer': 'Mikhail_Shogin',
  'issue_id': 'VR-6737',
  'summa

In [75]:
from collections import defaultdict
from datetime import datetime
import re

def escape_markdown(text):
    # Define the pattern for special Markdown characters that need to be escaped
    special_chars_pattern = r'([_*\[\]()~`>#+\-=|{}.!])'
    # Replace each special character with its escaped version
    escaped_text = re.sub(special_chars_pattern, r'\\\1', text)
    return escaped_text

mapped_user_names = {
    'Valentin_Marchuk': 'valuamba',
    'Vansevich.E': 'evansevich',
    'Mikhail_Shogin': 'mshogin'
}

grouped_issues = defaultdict(list)
for issue in review_items:
    grouped_issues[issue['reviewer']].append(issue)

# "[*" + escapeMarkdown(issueStr) + "*]"  + "(" + issueUrl + ")\n\n"

# Sort and format output
output = "\#review\n\nЗадачи на ревью:\n\n"
for reviewer, issues in grouped_issues.items():
    output += f"@{mapped_user_names[reviewer]}:\n"
    # Sort issues by hours_passed
    sorted_issues = sorted(issues, key=lambda x: x['hours_passed'], reverse=True)
    for issue in sorted_issues:
        issue_url = f'https://youtrack.wildberries.ru/issue/{issue["issue_id"]}'
        time_passed = f"{int(issue['hours_passed'] / 24)}d" if issue['hours_passed'] > 24 else f"{round(issue['hours_passed'])}h"
        output += f"{time_passed} [*\[{escape_markdown(issue['issue_id'])}\]*]({issue_url}) {escape_markdown(issue['summary'])}\n"
    output += '\n'

print(output.strip())

\#review

Задачи на ревью:

@valuamba:
34d [*\[VR\-8099\]*](https://youtrack.wildberries.ru/issue/VR-8099) \[BE\] Локальное развертывание сервисов crm\-api
6d [*\[VR\-8507\]*](https://youtrack.wildberries.ru/issue/VR-8507) \[BE\] Реализовать TokenExchange в crm\-api
1h [*\[VR\-8816\]*](https://youtrack.wildberries.ru/issue/VR-8816) \[BE\] Научиться запускать локально crm\-api

@evansevich:
172d [*\[VR\-5411\]*](https://youtrack.wildberries.ru/issue/VR-5411) \[BE\] \- AuthV3 \- Написать клиент для работы с auth v3
171d [*\[VR\-5474\]*](https://youtrack.wildberries.ru/issue/VR-5474) \[BE\] \- AuthV3 \- Интеграция клиента auth v3 в auth service
161d [*\[VR\-5417\]*](https://youtrack.wildberries.ru/issue/VR-5417) \[BE\] \- AuthV3 \- Интеграция auth v3 в competition\-api
110d [*\[VR\-6289\]*](https://youtrack.wildberries.ru/issue/VR-6289) \[BE\] Billing подпись\. Добавить подпись в balance
18d [*\[VR\-8093\]*](https://youtrack.wildberries.ru/issue/VR-8093) \[FE\] Переезд на новые кластера
7

<>:25: SyntaxWarning: invalid escape sequence '\#'
<>:33: SyntaxWarning: invalid escape sequence '\['
<>:33: SyntaxWarning: invalid escape sequence '\]'
<>:25: SyntaxWarning: invalid escape sequence '\#'
<>:33: SyntaxWarning: invalid escape sequence '\['
<>:33: SyntaxWarning: invalid escape sequence '\]'
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\3021288058.py:25: SyntaxWarning: invalid escape sequence '\#'
  output = "\#review\n\nЗадачи на ревью:\n\n"
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\3021288058.py:33: SyntaxWarning: invalid escape sequence '\['
  output += f"{time_passed} [*\[{escape_markdown(issue['issue_id'])}\]*]({issue_url}) {escape_markdown(issue['summary'])}\n"
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28028\3021288058.py:33: SyntaxWarning: invalid escape sequence '\]'
  output += f"{time_passed} [*\[{escape_markdown(issue['issue_id'])}\]*]({issue_url}) {escape_markdown(issue['summary'])}\n"


In [ ]:
https://t.me/c/2011635320/2/3

In [77]:
import asyncio
import telegram

test_chat = -1002011635320
test_thread = 2

# https://t.me/c/1593412877/10060/10061
bot = telegram.Bot("6779084548:AAEzWtAFQphlMonph8R-pw9IlU9YzFPal2k")


await bot.send_message(text=output.strip(), chat_id=-1001593412877, message_thread_id=10060, parse_mode='MarkdownV2')

Message(channel_chat_created=False, chat=Chat(id=-1001593412877, is_forum=True, title='HR INFRA TEAM', type=<ChatType.SUPERGROUP>), date=datetime.datetime(2024, 3, 11, 22, 13, 22, tzinfo=<UTC>), delete_chat_photo=False, entities=(MessageEntity(length=7, offset=0, type=<MessageEntityType.HASHTAG>), MessageEntity(length=9, offset=27, type=<MessageEntityType.MENTION>), MessageEntity(length=9, offset=42, type=<MessageEntityType.TEXT_LINK>, url='https://youtrack.wildberries.ru/issue/VR-8099'), MessageEntity(length=9, offset=42, type=<MessageEntityType.BOLD>), MessageEntity(length=9, offset=101, type=<MessageEntityType.TEXT_LINK>, url='https://youtrack.wildberries.ru/issue/VR-8507'), MessageEntity(length=9, offset=101, type=<MessageEntityType.BOLD>), MessageEntity(length=9, offset=155, type=<MessageEntityType.TEXT_LINK>, url='https://youtrack.wildberries.ru/issue/VR-8816'), MessageEntity(length=9, offset=155, type=<MessageEntityType.BOLD>), MessageEntity(length=11, offset=208, type=<MessageE